# models using `flask_sqlalchemy` + relationships + cascade

In [232]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

In [233]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'
db = SQLAlchemy(app)

# make classes (tables)

In [234]:
class Student(db.Model):
  __tablename__ = 'student'
  student_id = Column(Integer, primary_key=True, autoincrement=True)
  roll_number = Column(String, unique=True, nullable=False)
  first_name = Column(String, nullable=False)
  last_name = Column(String)

  enrolls = relationship('Enroll', back_populates='student', cascade='all, delete')


class Course(db.Model):
  __tablename__ = 'course'
  course_id = Column(Integer, primary_key=True, autoincrement=True)
  course_code = Column(String, unique=True, nullable=False)
  course_name = Column(String, nullable=False)
  course_description = Column(String)

  enrolls = relationship('Enroll', back_populates='course', cascade='all, delete')


class Enroll(db.Model):
  __tablename__ = 'enrollments'
  enrollment_id = Column(Integer, primary_key=True, autoincrement=True)
  estudent_id = Column(Integer, ForeignKey('student.student_id'), nullable=False)
  ecourse_id = Column(Integer, ForeignKey('course.course_id'), nullable=False)

  student = relationship('Student', back_populates='enrolls')
  course = relationship('Course', back_populates='enrolls')

# ~~make engine and session~~ `db.create_all()`

In [235]:
with app.app_context():
  db.create_all()

# populate data

In [236]:
with app.app_context():
  db.session.add(Course(course_code='CSE01', course_name='MAD 1', course_description='...'))
  db.session.add(Course(course_code='CSE02', course_name='DBMS', course_description='...'))
  db.session.add(Course(course_code='CSE03', course_name='PDSA', course_description='...'))
  db.session.add(Course(course_code='BST13', course_name='BDM', course_description='...'))
  db.session.add(Course(course_code='BST14', course_name='BDM2.0', course_description='...'))

  db.session.add(Student(roll_number='1', first_name='Harikesh'))
  db.session.add(Student(roll_number='2', first_name='Vidu'))

  db.session.add(Enroll(estudent_id=1, ecourse_id=1))
  db.session.add(Enroll(estudent_id=1, ecourse_id=4))
  db.session.add(Enroll(estudent_id=1, ecourse_id=5))
  db.session.add(Enroll(estudent_id=2, ecourse_id=1))
  db.session.add(Enroll(estudent_id=2, ecourse_id=2))
  db.session.add(Enroll(estudent_id=2, ecourse_id=4))
  db.session.add(Enroll(estudent_id=2, ecourse_id=5))

  # to delete
  db.session.add(Student(roll_number='3', first_name='Muskan'))
  db.session.add(Enroll(estudent_id=3, ecourse_id=1))
  db.session.add(Enroll(estudent_id=3, ecourse_id=2))

  db.session.commit()

# query data

Student.query == db.session.query(Student)

In [237]:
with app.app_context():
  bdm2 = Course.query.filter_by(course_id=5).first()
  if bdm2:
    db.session.delete(bdm2)
    db.session.commit()

In [238]:
with app.app_context():
  # Student.query.filter_by(student_id=3).delete()
  # db.session.query(Student).filter_by(student_id=3).delete()
  student = Student.query.filter_by(student_id=3).first()
  if student:
    db.session.delete(student)
    db.session.commit()

In [239]:
with app.app_context():
  students = Student.query.all()
  for student in students:
    print(student.first_name)

Harikesh
Vidu


In [240]:
with app.app_context():
  bdm = Course.query.filter_by(course_id=4).first()
  for enroll in bdm.enrolls:
    print(f'{enroll.student.first_name} has taken bdm')

Harikesh has taken bdm
Vidu has taken bdm


In [241]:
with app.app_context():
  enrolls = Enroll.query.all()
  for enroll in enrolls:
    # print(f'{enroll.estudent_id} {enroll.student}')
    print(f'{enroll.student.first_name} has taken {enroll.course.course_name}')

Harikesh has taken MAD 1
Harikesh has taken BDM
Vidu has taken MAD 1
Vidu has taken DBMS
Vidu has taken BDM


In [242]:
student_id = 1
with app.app_context():
  student = Student.query.filter_by(student_id=student_id).first()
  print(student)
  print(student.enrolls)
  print(student.enrolls[0])
  print(student.enrolls[0].course)
  print(student.enrolls[0].course.course_name)

<Student 1>
[<Enroll 1>, <Enroll 2>]
<Enroll 1>
<Course 1>
MAD 1


In [243]:
student_id = 1
with app.app_context():
  student = Student.query.filter_by(student_id=student_id).first()
  for enroll in student.enrolls:
    print(f'{student.first_name} has taken {enroll.course.course_name}')

Harikesh has taken MAD 1
Harikesh has taken BDM


In [244]:
student_id = 1
with app.app_context():
  student = Student.query.filter_by(student_id=student_id).first()
  enrolls = Enroll.query.filter_by(estudent_id=student_id).all()

  for enroll in enrolls:
    course = Course.query.filter_by(course_id=enroll.ecourse_id).first()
    print(f'{student.first_name} has taken course {course.course_name}')

Harikesh has taken course MAD 1
Harikesh has taken course BDM
